# Monster Chess — Play Against the AI

Run all setup cells first, then use the **Play** cell repeatedly. Each game auto-saves to `data/raw/human_games/`.

In [ ]:
# === Setup (run once) ===
import os, sys, json, time
import chess, chess.svg
from IPython.display import display, SVG, HTML, clear_output

from config import MODEL_DIR, RAW_DATA_DIR
from monster_chess import MonsterChessGame
from mcts import MCTS

# --- Configuration ---
PLAY_AS = "black"         # "black" or "white"
USE_HEURISTIC = True      # True = heuristic eval, False = neural network
MODEL_PATH = os.path.join(MODEL_DIR, "best_value_net.pt")
SIMULATIONS = 800         # MCTS search depth (higher = stronger + slower)

# Load evaluator
eval_fn = None
if not USE_HEURISTIC and os.path.exists(MODEL_PATH):
    from evaluation import NNEvaluator
    eval_fn = NNEvaluator(MODEL_PATH)
    print(f"Model loaded: {MODEL_PATH} (CUDA: {eval_fn.device.type == 'cuda'})")
else:
    print("Using heuristic evaluation")

engine = MCTS(num_simulations=SIMULATIONS, eval_fn=eval_fn)
print(f"Playing as {PLAY_AS.upper()} | {SIMULATIONS} simulations")

In [ ]:
# === Helper functions (run once) ===

def show_board(board, last_move=None, flip=False, size=480):
    """Display the board as SVG with highlighted last move."""
    arrows = []
    squares = {}
    highlight_color = "#e8e846"

    if last_move:
        if isinstance(last_move, tuple):
            m1, m2 = last_move
            # First move in blue, second in green
            arrows.append(chess.svg.Arrow(m1.from_square, m1.to_square, color="#4a90d9aa"))
            if m2 != chess.Move.null():
                arrows.append(chess.svg.Arrow(m2.from_square, m2.to_square, color="#50c878aa"))
        else:
            arrows.append(chess.svg.Arrow(last_move.from_square, last_move.to_square, color="#4a90d9aa"))

    svg = chess.svg.board(
        board,
        arrows=arrows,
        fill=squares,
        flipped=flip,
        size=size,
        coordinates=True,
    )
    display(SVG(svg))


def show_eval_bar(value):
    """Show an HTML eval bar."""
    pct = int((value + 1) / 2 * 100)
    pct = max(2, min(98, pct))
    if value > 0.3:
        label = f"<b>+{value:.2f}</b> White"
    elif value < -0.3:
        label = f"<b>{value:.2f}</b> Black"
    else:
        label = f"{value:.2f} Even"
    html = f"""
    <div style="width:480px;height:24px;border:1px solid #888;border-radius:4px;overflow:hidden;display:flex;margin:4px 0">
      <div style="width:{pct}%;background:#f0f0f0"></div>
      <div style="width:{100-pct}%;background:#333"></div>
    </div>
    <div style="font-family:monospace;font-size:14px;margin-bottom:8px">{label}</div>
    """
    display(HTML(html))


def parse_move(text, board):
    """Parse UCI (e2e4) or SAN (Nf3) into a chess.Move."""
    text = text.strip()
    try:
        move = chess.Move.from_uci(text)
        if move in board.pseudo_legal_moves or move in board.legal_moves:
            return move
    except (ValueError, chess.InvalidMoveError):
        pass
    try:
        return board.parse_san(text)
    except (ValueError, chess.InvalidMoveError, chess.AmbiguousMoveError):
        return None


def save_game(records, result):
    """Save game records to human_games directory."""
    for rec in records:
        rec["game_result"] = result
    out_dir = os.path.join(RAW_DATA_DIR, "human_games")
    os.makedirs(out_dir, exist_ok=True)
    existing = len([f for f in os.listdir(out_dir) if f.endswith(".jsonl")])
    path = os.path.join(out_dir, f"game_{existing:05d}.jsonl")
    with open(path, "w") as f:
        for rec in records:
            f.write(json.dumps(rec) + "\n")
    return path, len(records)


print("Helpers loaded.")

## Play a Game

Run this cell for each new game. Enter moves as **UCI** (`e7e5`, `b8c6`) or **SAN** (`e5`, `Nc6`). Type `quit` to resign.

In [ ]:
# === PLAY A GAME (run this cell for each new game) ===
human_is_white = (PLAY_AS == "white")
flip = human_is_white
game = MonsterChessGame()
records = []
move_number = 0

print("=== NEW GAME ===")
print(f"You: {PLAY_AS.upper()} | AI: {'BLACK' if human_is_white else 'WHITE (double moves)'}\n")
show_board(game.board, flip=flip)

resigned = False
while not game.is_terminal():
    is_white = game.is_white_turn
    is_human = (is_white == human_is_white)

    if is_human:
        if human_is_white:
            # --- Human plays White (double move) ---
            game.board.turn = chess.WHITE
            legal = list(game.board.pseudo_legal_moves)
            while True:
                text = input("Your move 1/2: ").strip()
                if text.lower() in ("quit", "resign", "q"):
                    resigned = True; break
                m1 = parse_move(text, game.board)
                if m1 and m1 in legal:
                    break
                print(f"  Invalid. Try: {', '.join(m.uci() for m in legal[:15])}...")

            if resigned: break

            game.board.push(m1)
            if game.board.king(chess.BLACK) is None:
                game.board.pop()
                action = (m1, chess.Move.null())
            else:
                clear_output(wait=True)
                print(f"Move 1: {m1.uci()}")
                show_board(game.board, last_move=m1, flip=flip)
                game.board.turn = chess.WHITE
                legal2 = list(game.board.pseudo_legal_moves)
                while True:
                    text = input("Your move 2/2: ").strip()
                    if text.lower() in ("quit", "resign", "q"):
                        game.board.pop(); resigned = True; break
                    m2 = parse_move(text, game.board)
                    if m2 and m2 in legal2:
                        break
                    print(f"  Invalid. Try: {', '.join(m.uci() for m in legal2[:15])}...")
                if resigned: break
                game.board.turn = chess.BLACK
                game.board.pop()
                action = (m1, m2)
        else:
            # --- Human plays Black (single move) ---
            game.board.turn = chess.BLACK
            legal = list(game.board.legal_moves)
            if not legal:
                print("No legal moves! Stalemate.")
                break
            while True:
                text = input("Your move: ").strip()
                if text.lower() in ("quit", "resign", "q"):
                    resigned = True; break
                move = parse_move(text, game.board)
                if move and move in legal:
                    action = move; break
                print(f"  Invalid. Try: {', '.join(m.uci() for m in legal[:15])}...")
            if resigned: break

        game.apply_action(action)
        clear_output(wait=True)
        print(f"You played: {action[0].uci()} + {action[1].uci()}" if human_is_white else f"You played: {action.uci()}")
        show_board(game.board, last_move=action, flip=flip)

    else:
        # --- AI turn ---
        side = "White" if is_white else "Black"
        print(f"AI ({side}) thinking...", end="", flush=True)
        t0 = time.time()
        action, action_probs, root_value = engine.get_best_action(game, temperature=0.1)
        elapsed = time.time() - t0

        if action is None:
            print(" No legal moves!")
            break

        # Record for training
        records.append({
            "fen": game.fen(),
            "mcts_value": round(root_value, 4),
            "policy": action_probs,
            "current_player": "white" if is_white else "black",
        })

        game.apply_action(action)

        if is_white:
            m1, m2 = action
            move_str = f"{m1.uci()} + {m2.uci()}"
        else:
            move_str = action.uci()

        clear_output(wait=True)
        print(f"AI ({side}): {move_str}  ({elapsed:.1f}s)")
        show_board(game.board, last_move=action, flip=flip)
        show_eval_bar(root_value)

    move_number += 1

# --- Game over ---
if resigned:
    result = -1 if human_is_white else 1
    winner = "Black" if human_is_white else "White"
    print(f"\nYou resigned. {winner} wins!")
elif game.is_terminal():
    result = game.get_result()
    if result > 0:
        print("\nWhite wins! (king captured)")
    elif result < 0:
        print("\nBlack wins! (king captured)")
    else:
        print("\nDraw (turn limit)")
else:
    result = 0

# Auto-save
if records:
    path, n = save_game(records, result)
    print(f"Game saved: {path} ({n} positions)")